In [1]:
import re
import json
import csv
import requests
import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chisquare
from collections import Counter, defaultdict

In [2]:
data_root = '/home/haopeng/Data/Retraction/'

### Link based on doi

In [3]:
data = pd.read_excel(data_root+'RetractionWatchData2020.xlsx', sheet_name='GridViewExport (22)', header=0, index_col = None)


In [4]:
data.head(1)

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes
0,23540,Laquinimod Inhibits MMP-induced NLRP3 Inflamma...,(BLS) Biochemistry;(BLS) Biology - Cellular;(B...,"department of Neurology, Affiliated Hosptial o...",Immunopharmacology and Immunotoxicology,Taylor and Francis,China,Xue Zhang;Jianjing Jin;Anmu Xie,NaN,Research Article;,2020-06-05 00:00:00,10.1080/08923973.2020.1774840,32500758,2020-04-05 00:00:00,10.1080/08923973.2020.1746967,32249647,Retraction,+Error in Analyses;,No,NaN


In [5]:
data.shape

(21850, 20)

In [11]:
data.loc[data.OriginalPaperDOI=='10.1105/tpc.020719']

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes
7715,2719,Probing the MicroRNA and Small Interfering RNA...,(BLS) Biology - Cellular;(BLS) Biology - Molec...,Institut de Biologie Moléculaire des Plantes d...,The Plant Cell,American Society of Plant Biologists,France,Patrice Dunoyer;Charles-Henri Lecellier;Eneida...,http://retractionwatch.com/2015/04/06/retracti...,Research Article;,2015-06-02 00:00:00,10.1105/tpc.15.00407,26036255,2004-04-14 00:00:00,10.1105/tpc.020719,15084715,Retraction,+Duplication of Image;+Investigation by Compan...,No,Patrice Dunoyer assumes responsibility for fig...


In [16]:
data.loc[data.OriginalPaperDOI=='10.1111/gbb.12373']

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes
4547,12734,Nrf2: a novel therapeutic target in fragile X ...,(BLS) Biochemistry;(BLS) Biology - Cellular;(B...,"Biomedicine Division, Centre for Systems Biote...","Genes, Brain, and Behavior",Wiley,Argentina;Chile;United Kingdom;United States,Robert M J Deacon;M J Hurley;C M Rebolledo;M S...,http://retractionwatch.com/2017/10/11/dispute-...,Research Article;,2017-09-01 00:00:00,10.1111/gbb.12373,28211606,2017-02-17 00:00:00,10.1111/gbb.12373,28211606,Retraction,+Complaints about Author;+Concerns/Issues Abou...,No,NaN


In [56]:
df = data.loc[~data['OriginalPaperDOI'].isna()]
df = df.loc[~df['OriginalPaperDOI'].isin(['Unavailable', 'unavailable'])]
df.OriginalPaperDOI = df['OriginalPaperDOI'].apply(lambda x: x.lower())
df = df.drop_duplicates(subset=['OriginalPaperDOI'], keep = 'first')
df.index = range(len(df))

In [57]:
len(df)

16460

In [58]:
dois = set(df['OriginalPaperDOI'].tolist())

In [59]:
len(dois)

16460

Alt data doc: https://help.altmetric.com/support/solutions/articles/6000086844-sample-response-for-the-altmetric-details-page-api-full-access-

In [ ]:
# A no_doi_field example:

{"altmetric_id":27560509,"citation":{"aggregate_citation_ids":[],"attribution":"Google Books","authors_or_editors":["Peter J. McGoldrick"],"first_seen_on":"2017-10-17T15:41:45+00:00","isbns":["9780077092504"],"issns":[],"links":["https:\/\/books.google.com\/books?vid=9780077092504"],"pubdate":"2002-01-01T00:00:00+00:00","publisher":"McGraw-Hill","title":"Retail Marketing","type":"book","uri":"http:\/\/books.google.com\/books?id=cH52QgAACAAJ&hl=&source=gbs_api"},"altmetric_score":{"score":0,"score_history":{"1y":0,"6m":0,"3m":0,"1m":0,"1w":0,"6d":0,"5d":0,"4d":0,"3d":0,"2d":0,"1d":0,"at":0},"context_for_score":null},"demographics":[],"counts":{"total":{"posts_count":0}},"posts":[]}

# A bad line example:

{"altmetric_id":11743682,"counts":{"readers":{"mendeley":20,"citeulike":0,"connotea":0},"total":{"posts_count":0}},"citation":{"altmetric_jid":"4f6fa4ef3cf058f610002cc7","authors":["Kristina M. Cook","H. Patrick McNeil","Philip J. Hogg"],"doi":"10.1074\/jbc.m113.523506","endpage":"34929","first_seen_on":"2016-09-05T04:54:15+00:00","handles":[],"isbns":[],"issns":["0021-9258","1083-351X"],"issue":"48","journal":"Journal of Biological Chemistry","pmid":"24142694","pubdate":"2013-11-29T00:00:00+00:00","epubdate":"2013-10-18T00:00:00+00:00","publisher_subjects":[{"name":"Biological Sciences","scheme":"era"},{"name":"Medical And Health Sciences","scheme":"era"},{"name":"Chemical Sciences","scheme":"era"}],"scopus_subjects":["Biochemistry, Genetics and Molecular Biology","Life Sciences"],"startpage":"34920","subjects":["biochemistry"],"title":"Allosteric Control of \u03b2II-Tryptase by a Redox Active Disulfide Bond","type":"article","volume":"288","mendeley_url":"http:\/\/www.mendeley.com\/research\/allosteric-control-%CE%B2iitryptase-redox-active-disulfide-bond"},"altmetric_score":{"score":0,"score_history":{"1y":0,"6m":0,"3m":0,"1m":0,"1w":0,"6d":0,"5d":0,"4d":0,"3d":0,"2d":0,"1d":0,"at":0},"context_for_score":null},"demographics":{"users":{"mendeley":{"by_status":{"Unspecified":4,"Professor > Associate Professor":1,"Researcher":6,"Student  > Ph. D. Student":5,"Student  > Master":2,"Student  > Bachelor":1,"Professor":1},"by_discipline":{"Engineering":1,"Medicine and Dentistry":2,"Chemistry":1,"Agricultural and Biological Sciences":7,"Computer Science":1,"Biochemistry, Genetics and Molecular Biology":2,"Unspecified":6}}},"geo":{"mendeley":{"CA":1,"JP":1}}},"posts":[]}{"altmetric_id":7751729,"counts":{"readers":{"mendeley":11,"citeulike":0,"connotea":0},"total":{"posts_count":0}},"citation":{"altmetric_jid":"4f6fa5163cf058f610003742","authors":["D. M. Bolton","A. Ta","M. Bagnato","D. Muller","N. L. Lawrentschuk","G. Severi","R. R. Syme","G. G. Giles"],"doi":"10.1007\/s00345-013-1125-0","endpage":"435","first_seen_on":"2016-05-19T21:14:51+00:00","handles":[],"isbns":[],"issns":["0724-4983","1433-8726"],"issue":"2","journal":"World Journal of Urology","pmid":"23824175","pubdate":"2014-04-01T00:00:00+00:00","epubdate":"2013-07-04T00:00:00+00:00","publisher_subjects":[{"name":"Medicine & Public Health","scheme":"springer"},{"name":"Nephrology","scheme":"springer"},{"name":"Oncology","scheme":"springer"},{"name":"Urology\/Andrology","scheme":"springer"},{"name":"Clinical Sciences","scheme":"era"}],"scopus_subjects":["Medicine","Health Sciences"],"startpage":"431","title":"Interval to biochemical recurrence following radical prostatectomy does not affect survival in men with low-risk prostate cancer","type":"article","uri":"http:\/\/link.springer.com\/10.1007\/s00345-013-1125-0","volume":"32","mendeley_url":"http:\/\/www.mendeley.com\/research\/interval-biochemical-recurrence-following-radical-prostatectomy-not-affect-survival-men-lowrisk-pros"},"altmetric_score":{"score":0,"score_history":{"1y":0,"6m":0,"3m":0,"1m":0,"1w":0,"6d":0,"5d":0,"4d":0,"3d":0,"2d":0,"1d":0,"at":0},"context_for_score":null},"demographics":{"users":{"mendeley":{"by_status":{"Unspecified":2,"Professor > Associate Professor":1,"Researcher":2,"Student  > Doctoral Student":2,"Student  > Ph. D. Student":2,"Student  > Master":1,"Student  > Bachelor":1},"by_discipline":{"Medicine and Dentistry":9,"Unspecified":2}}}},"posts":[]}


In [ ]:
# before running this code: combine all individual files in `oct-8th-dump.tar.gz` provided by Altmetric into `merged.txt` (can use Linux command line tool).

paper_li = []
bad_lines = 0
cn_lines = 0
no_doi_field = 0 
empty_doi = 0

with open(data_root + 'merged.txt', 'r') as ofile:
    for row in ofile:
        cn_lines += 1
        if cn_lines % 1000000 == 0:
            print('processed %d lines...'%cn_lines)
        if '}{"altmetric_id"' in row:
            bad_lines += 1
            row = row.replace('}{"altmetric_id"', '}\n{"altmetric_id"')
        # this way works for both good and bad lines.
        records = row.split('\n')
        for record in records:
            if record != '':
                paper = json.loads(record)
                if 'doi' in paper['citation']:
                    doi = paper['citation']['doi']
                    if doi != '':
                        doi = doi.lower()
                        if doi in dois:
                            paper_li.append(paper)
                    else:
                        empty_doi += 1
                else:
                    no_doi_field += 1

In [229]:
print('%d bad formatted json lines are successfully processed.'%bad_lines)
print('%d records have no doi field.'%no_doi_field) # e.g., books that only have ISBN.
print('%d records have doi field, but are empty strings.'%empty_doi)

514171 bad formatted json lines are successfully processed.
4139724 records have no doi field.
30 records have doi field, but are empty strings.


In [209]:
len(paper_li)

6840

In [254]:
with open(data_root+"retraction_altmetric.json", 'w') as ffile:
    for paper in paper_li:
        ffile.write(json.dumps(paper) + '\n')

Clean matched data from Altmetric (doi has different altids)

In [18]:
# records = []
# matched_dois = []

# with open(data_root+"retraction_altmetric.json", 'r') as ffile:
#     for row in ffile:
#         record = json.loads(row)
#         record['citation']['doi'] = record['citation']['doi'].lower()
#         records.append(record)
#         matched_dois.append(record['citation']['doi'])

In [24]:
# dup_dois = [doi for doi, count in Counter(matched_dois).items() if count > 1]

In [86]:
# for record in paper_li:
#     if record['citation']['doi'] in dup_dois:
#         pprint.pprint(record, indent=2)

In [27]:
# bad_alt_ids = [4406258, 67961220]

In [28]:
# paper_li = []
# for record in records:
#     if record['altmetric_id'] not in bad_alt_ids: 
#         paper_li.append(record)

In [30]:
# del records, matched_dois

In [254]:
with open(data_root+"retraction_altmetric.json", 'w') as ffile:
    for paper in paper_li:
        ffile.write(json.dumps(paper) + '\n')

In [31]:
paper_li = []

with open(data_root+"retraction_altmetric.json", 'r') as ffile:
    for row in ffile:
        record = json.loads(row)
        record['citation']['doi'] = record['citation']['doi'].lower()
        paper_li.append(record)

In [32]:
# num. of papers found in Alt.
len(paper_li)

6838